In [ ]:
import selenium
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get('https://www.youtube.com/@HYBELABELS/about')
sub_num=driver.find_element_by_xpath('//*[@id="subscriber-count"]').text
channel_name=driver.find_element_by_xpath('//*[@id="text"]').text
creation_date=driver.find_element_by_xpath('//*[@id="right-column"]/yt-formatted-string[2]/span[2]').text
total_view=driver.find_element_by_xpath('//*[@id="right-column"]/yt-formatted-string[3]').text
explanation=driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-channel-about-metadata-renderer/div[1]/div[1]/yt-formatted-string[2]').text
time.sleep(2)
driver.find_element_by_xpath('//*[@id="tabsContent"]/tp-yt-paper-tab[2]/div').click()


import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
you = db.cursor()


sql = """
    create table youtube(
        channel_name varchar(50) not null,
        creation_date date not null,
        sub_num bigint,
        total_view bigint,
        video_num bigint,
        primary key(channel_name)
    );
"""
you.execute(sql)
creation_date=creation_date[:-1].replace('. ','-')
sub_num = int(sub_num[4:-2])*10000
total_view = int(total_view[4:-1].replace(',',''))
video_num = 1006

sql = """
insert into youtube(channel_name, creation_date, sub_num, total_view, video_num) 
values ("{0}", "{1}", {2}, {3}, {4});
""".format(channel_name, creation_date, sub_num, total_view, video_num)
you.execute(sql)

sql = """
    create table video(
    channel_name char(20) not null,
    title varchar(100),
    views bigint,
    likes bigint,
    postdate date
    );
"""
you.execute(sql)







#하나하나 클릭하면서 긁어오는 부분

for i in range(1, 337):
    for j in range(1, 4):
        try:
            while True:
                prev_height = driver.execute_script('return document.documentElement.scrollHeight')
                driver.execute_script('window,scrollTo(0, document.documentElement.scrollHeight)')
                time.sleep(1)
                curr_height = driver.execute_script('return document.documentElement.scrollHeight')
                if curr_height==prev_height:
                    break
            driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[6]/ytd-rich-grid-row[{0}]/div/ytd-rich-item-renderer[{1}]'.format(i, j)).click()
            driver.find_element_by_xpath('//*[@id="expand"]').click()
            title = driver.find_element_by_xpath('//*[@id="title"]/h1/yt-formatted-string').text
            views = driver.find_element_by_xpath('//*[@id="info"]/span[1]').text
            postdate = driver.find_element_by_xpath('//*[@id="info"]/span[3]').text
            likes = driver.find_element_by_xpath('//*[@id="segmented-like-button"]/ytd-toggle-button-renderer/yt-button-shape/button/div[2]/span').text

            views = views[4:-1].replace(',', '')
            likes = likes.replace('만', '0000')
            likes = likes.replace('천', '000')
            likes = likes.replace('.', '')
            if "최초" in postdate:
                postdate = postdate[7:]
            postdate = postdate[:-1].replace('. ', '-')
            views = int(views)
            likes = int(likes)
            sql = """
                    insert into video(channel_name, title, views, likes, postdate)
                    values ("{0}", '''{1}''', {2}, {3}, "{4}");
                    """.format(channel_name, title, views, likes, postdate)
            you.execute(sql)
            db.commit()
            driver.back()
            time.sleep(1)
        except Exception as e:
            print(e)
            continue